Image classification with CNNs
================

The goal of this exercise is to implement a specific CNN architecture with PyTorch and train it on the CIFAR-10 image classification dataset. We will start by introducing the dataset and then implement a `nn.Module` and a useful `Solver` class. Seperating the model from the actual training has proven itself as a sensible design decision. By the end of this exercise you should have succesfully trained your (possible) first CNN model and have a boilerplate `Solver` class which you can reuse for the next exercise and your future research projects.

For an inspiration on how to implement a model or the solver class you can have a look at [these](https://github.com/pytorch/examples) PyTorch examples.

In [8]:
import numpy as np
import os
from random import choice
from string import ascii_uppercase
import torch
from torch.autograd import Variable
from yz.data_utils_harddisk import get_Cancer_datasets
from yz.solver import Solver
from yz.data_utils_harddisk import get_balanced_weights
from torchvision import models
import torch.nn as nn
import pandas as pd
from bayes_opt import BayesianOptimization
from tqdm import tqdm

csv_full_name = '~/dl4cvproject/data/train.csv'
img_folder_full_name = '~/dl4cvproject/data/train256'
csv_full_name = os.path.expanduser(csv_full_name)
img_folder_full_name = os.path.expanduser(img_folder_full_name)

csv_full_name_test = '~/dl4cvproject/data/test.csv'
img_folder_full_name_test = '~/dl4cvproject/data/test256'
csv_full_name_test = os.path.expanduser(csv_full_name_test)
img_folder_full_name_test = os.path.expanduser(img_folder_full_name_test)

#%matplotlib inline
#plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
#plt.rcParams['image.interpolation'] = 'nearest'
#plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
train_data, val_data, test_data, train_label_list = get_Cancer_datasets(csv_full_name=csv_full_name,img_folder_full_name=img_folder_full_name)
test_X, csv_test = get_Cancer_datasets(csv_full_name=csv_full_name_test,img_folder_full_name=img_folder_full_name_test, mode='upload')
print("Train size: %i" % len(train_data))
print("Val size: %i" % len(val_data))
print("Test size: %i" % len(test_data))
print("upload size: {}".format(len(test_X)))


100%|██████████| 18577/18577 [00:00<00:00, 912309.12it/s]

num_training:13000
num_validation:1857
num_test:3720
OK...
Train size: 13000
Val size: 1857
Test size: 3719
upload size: 12386


In [10]:
if torch.cuda.is_available():
    print('Cuda available')
else:
    print('Cuda not available :(---(')

Cuda available


In [11]:
def target(factor, batch_size, lr_const, lr_exp, weight_decay_const, weight_decay_exp, num_epochs):

    batch_size = int(batch_size)
    num_epochs = int(num_epochs)
    lr_const = int(lr_const)
    weight_decay_const = int(weight_decay_const)
    
    #training
    weights = get_balanced_weights(label_list=train_label_list, num_classes=14, factor=factor)
    sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False, sampler=sampler, num_workers=8)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=20, shuffle=False, num_workers=8)
    
    model = models.resnet34(pretrained=True)
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, 14)

    lr = lr_const * np.power(10, lr_exp)
    weigth_decay = weight_decay_const * np.power(10, weight_decay_exp)    
    solver = Solver(optim_args={"lr":lr, "weight_decay":weigth_decay})
    solver.train(model, train_loader, val_loader, log_nth=50, num_epochs=num_epochs)
    
    #compute local prediction acc
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=30, shuffle=False, num_workers=8)
    scores = []
    for inputs, target in tqdm(test_loader):
        inputs, targets = Variable(inputs), Variable(target)
        if torch.cuda.is_available:
            inputs, targets = inputs.cuda(), targets.cuda()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        scores.extend((preds == targets).data.cpu().numpy())
        
    test_acc = np.mean(scores)
    print(test_acc)
    
    ## generate submission file: submissions/res18_acc_randomsuffix.csv
    if test_acc >= 0.32:
        
        file_name = 'submissions/res18_' + '{:.5f}'.format(test_acc) + '_' + ''.join(choice(ascii_uppercase) for i in range(7)) + '.csv'
        print(file_name)
        
        try:
            del csv_test['age']
        except KeyError as e:
            print(e)
        try:
            del csv_test['gender']
        except KeyError as e:
            print(e)
        try:
            del csv_test['view_position']
        except KeyError as e:
            print(e)
        try:
            del csv_test['image_name']
        except KeyError as e:
            print(e)
        try:
            del csv_test['detected']
        except KeyError as e:
            print(e)

        pred_set = set()
        detected = []
        for i in tqdm(range(len(test_X))):
            tmp_pred_list = [0] * 14
            inputs = test_X[i]
            inputs = Variable(inputs.unsqueeze(0))
            if torch.cuda.is_available:
                inputs = inputs.cuda()
            for trial in range(1):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                int_label = preds.data.cpu().numpy().tolist()[0]
                tmp_pred_list[int_label] += 1

            pred = tmp_pred_list.index(max(tmp_pred_list))
            str_pred = 'class_' + str(pred + 1)
            detected.append(str_pred)

        csv_test['detected'] = pd.Series(detected)
        csv_test.to_csv(file_name, index=False)
    
    return test_acc

    

## Bayesian Optimization

In [12]:
bo = BayesianOptimization(target, {'factor':(0.5, 1), 'batch_size':(30, 80),
                                   'lr_const':(1, 10), 'lr_exp':(-7, -3),
                                   'weight_decay_const':(1, 10), 'weight_decay_exp':(-6, -1),
                                   'num_epochs':(2, 8)})

In [ ]:
bo.maximize(init_points=2, n_iter=1000, acq='ucb', kappa=5)

Initialization
--------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   batch_size |    factor |   lr_const |    lr_exp |   num_epochs |   weight_decay_const |   weight_decay_exp | 
weights: [0.0066083103160308257, 0.0044707798770110708, 0.00059174179826106826, 0.0024360986978506543, 0.0013595489025720131, 0.0023203019996160972, 0.0012863210206126134, 0.007336018753684722, 0.036364718182485108, 0.013522111523254931, 0.0037440046433332608, 0.0018467992779463438, 0.0042878268781592135, 0.0053994175869958522]
equivalent_num:
2.00231802576
2.05655874343
2.36164151686
2.14376685411
2.23101974912
2.15091995364
2.23948489689
1.98806108225
1.78187119094
1.90661772478
2.08166658169
2.18476354581
2.06244472839
2.03018101271


Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /home/ubuntu/.torch/models/resnet34-333f7ec4.pth

 43%|████▎     | 37781504/87306240 [00:00<00:00, 94771121.54it/s]
Exception in thread Thread-6:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/ubuntu/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

  0%|          | 0/2 [00:00<?, ?it/s]

START TRAIN.
[Iteration 1 / 636] TRAIN loss: 2.748323
[Iteration 51 / 636] TRAIN loss: 2.405894
[Iteration 101 / 636] TRAIN loss: 2.239165
[Iteration 151 / 636] TRAIN loss: 2.233663
[Iteration 201 / 636] TRAIN loss: 2.111372
[Iteration 251 / 636] TRAIN loss: 1.897411
[Iteration 301 / 636] TRAIN loss: 1.891078
[Epoch 1 / 2] TRAIN acc: 11.484277


 50%|█████     | 1/2 [03:41<03:41, 221.75s/it]

[Epoch 1 / 2] VAL acc: 4.236559
[Iteration 351 / 636] TRAIN loss: 1.751199
[Iteration 401 / 636] TRAIN loss: 1.933808
[Iteration 451 / 636] TRAIN loss: 2.046303
[Iteration 501 / 636] TRAIN loss: 1.882451
[Iteration 551 / 636] TRAIN loss: 1.892066
[Iteration 601 / 636] TRAIN loss: 1.498814
[Epoch 2 / 2] TRAIN acc: 15.034591


...

[Epoch 5 / 8] VAL acc: 1.591398
[Iteration 1551 / 2424] TRAIN loss: 2.699441
[Iteration 1601 / 2424] TRAIN loss: 2.708267
[Iteration 1651 / 2424] TRAIN loss: 2.743096
[Iteration 1701 / 2424] TRAIN loss: 2.672343
[Iteration 1751 / 2424] TRAIN loss: 2.690572
[Iteration 1801 / 2424] TRAIN loss: 2.681288
[Epoch 6 / 8] TRAIN acc: 3.336634


 75%|███████▌  | 6/8 [21:47<07:15, 217.96s/it]

[Epoch 6 / 8] VAL acc: 1.666667
[Iteration 1851 / 2424] TRAIN loss: 2.670452
[Iteration 1901 / 2424] TRAIN loss: 2.783732
[Iteration 1951 / 2424] TRAIN loss: 2.791209
[Iteration 2001 / 2424] TRAIN loss: 2.726537
[Iteration 2051 / 2424] TRAIN loss: 2.698045
[Iteration 2101 / 2424] TRAIN loss: 2.714332
[Epoch 7 / 8] TRAIN acc: 3.231023


 88%|████████▊ | 7/8 [25:25<03:37, 217.95s/it]

[Epoch 7 / 8] VAL acc: 1.688172
[Iteration 2151 / 2424] TRAIN loss: 2.727952
